# `metadsl`

...is a framework for creating domain specific languages in Python.

In [1]:
from __future__ import annotations

from metadsl import *
from metadsl_core import *
from metadsl_visualize import *

Here, we show a NumPy API compatible DSL that we have started writing. We can create a symbolic expression and then execute it. The widget shows the different stages of execution:

In [2]:
N = 100
expr = (arange(N) + arange(N))[5].to_ndarray()
expr

Typez(definitions=None, nodes={'-3705275237864060643': PrimitiveNode(type='int', repr='100'), '-7055281088629370118': CallNode(function='metadsl_core.numpy.arange', type_params=None, args=['-3705275237864060643'], kwargs=None), '5764860230719714798': CallNode(function='metadsl_core.numpy.NDArrayCompat.__add__', type_params=None, args=['-7055281088629370118', '-7055281088629370118'], kwargs=None), '-3705275237759055718': PrimitiveNode(type='int', repr='5'), '6550727564280151447': CallNode(function='metadsl_core.numpy.NDArrayCompat.__getitem__', type_params=None, args=['5764860230719714798', '-3705275237759055718'], kwargs=None), '-2038961938848683980': CallNode(function='metadsl_core.numpy.NDArrayCompat.to_ndarray', type_params=None, args=['6550727564280151447'], kwargs=None), '-2927105886577020080': CallNode(function='metadsl_core.conversion.Converter.convert', type_params={'T': DeclaredTypeInstance(type='metadsl_core.numpy.NDArray', params=None)}, args=['5764860230719714798'], kwargs=

10

Now, we can add an optimization replacement, so that we do the getitem before adding the two expressions:

In [3]:
@register
@rule
def optimize_getitem_add(l: NDArray, r: NDArray, idx: IndxType) -> R[NDArray]:
    return (
        # expression to match against
        (l + r)[idx],
        # expression to replace it with
        l[idx] + r[idx]
    )

Now, when we look at the execution steps, we see that it is doing each getitem before adding, which should be more efficient:

In [4]:
expr = (arange(N) + arange(N))[5].to_ndarray()
expr

Typez(definitions=None, nodes={'-3705275237864060643': PrimitiveNode(type='int', repr='100'), '-7055281088629370118': CallNode(function='metadsl_core.numpy.arange', type_params=None, args=['-3705275237864060643'], kwargs=None), '5764860230719714798': CallNode(function='metadsl_core.numpy.NDArrayCompat.__add__', type_params=None, args=['-7055281088629370118', '-7055281088629370118'], kwargs=None), '-3705275237759055718': PrimitiveNode(type='int', repr='5'), '6550727564280151447': CallNode(function='metadsl_core.numpy.NDArrayCompat.__getitem__', type_params=None, args=['5764860230719714798', '-3705275237759055718'], kwargs=None), '-2038961938848683980': CallNode(function='metadsl_core.numpy.NDArrayCompat.to_ndarray', type_params=None, args=['6550727564280151447'], kwargs=None), '-2927105886577020080': CallNode(function='metadsl_core.conversion.Converter.convert', type_params={'T': DeclaredTypeInstance(type='metadsl_core.numpy.NDArray', params=None)}, args=['5764860230719714798'], kwargs=

10

This shows how we can break up the NumPy API into different layers, all of which are extensible:

1. A compatibility layer that works like the existing NumPy API, except isn't limited to the Python types of the current API
2. A type safe version of this API. The conversion between the compatability layer and this layer is extensible, so that third party authors can add new conversion between their own Python objects and the typed representation.
3. A backend layer that translates either back to Python calls or source code, or to other targets like LLVM or Tensorflow.

The key is that all these layers are composable, so you could have different frontends for any of them or add your own. This is all done through a typed replacement system that is compatible with static analysis using MyPy.

`metadsl` is the glue that allows us to define this API in a way that enables greater greater collaboration. Since each layer has it's own well defined specification, different projects can extend and target these seperate layers, while minimizing the need to explicitly opt in to collaborate. The goal is to enable greater reuse, interoperability, and extensability within the Python scientific ecosystem.